In [1]:
#imports
import pandas as pd
import gc
import re
import numpy as np

import warnings 
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler
import torch
import datasets
import transformers
print(transformers.__version__)

4.28.1


In [2]:
RATE_HZ = 16000 # resampling rate in Hz
MAX_LENGTH = 160000 # maximum audio interval length to consider (= RATE_HZ * SECONDS)
labels = ['cat', 'dog']
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

print(id2label, '\n\n', label2id)

{0: 'cat', 1: 'dog'} 

 {'cat': 0, 'dog': 1}


In [3]:
import torchaudio
print(torchaudio.__version__)

0.12.0+cu113


In [4]:
from pathlib import Path

lang_counter = dict()

def load_data():
    file_list = []
    label_list = []
    for file in Path('cats_dogs').glob('*/*/*.wav'):
        lang = re.match(r"([a-z]+)", file.stem).group(1)
        file_list.append(file)
        label_list.append(label2id[lang])
    dd = pd.DataFrame()
    dd['file'] = file_list
    dd['label'] = label_list
    return dd

In [5]:
%%time
dd = load_data()
dd.sample(5)

Wall time: 11.7 ms


,file,label
20,cats_dogs\test\cats\cat_3.wav,0
90,cats_dogs\train\cat\cat_124.wav,0
232,cats_dogs\train\dog\dog_barking_4.wav,1
276,cats_dogs\train\dog\dog_barking_98.wav,1
261,cats_dogs\train\dog\dog_barking_79.wav,1


In [6]:
dd['label'].value_counts()

0    164
1    113
Name: label, dtype: int64

In [7]:
def get_transform_audio(file):
    audio,rate = torchaudio.load(str(file))
    transform = torchaudio.transforms.Resample(rate,RATE_HZ)
    audio = transform(audio).squeeze(0).numpy()
    audio = audio[:MAX_LENGTH] # truncate to first part of audio to save RAM
    return audio
dd['audio'] = dd['file'].progress_apply(get_transform_audio)

100%|██████████| 277/277 [00:00<00:00, 1108.85it/s]


In [9]:
print(dd)

                                       file  label  \
0           cats_dogs\test\cats\cat_110.wav      0   
1           cats_dogs\test\cats\cat_112.wav      0   
2           cats_dogs\test\cats\cat_115.wav      0   
3           cats_dogs\test\cats\cat_126.wav      0   
4           cats_dogs\test\cats\cat_129.wav      0   
..                                      ...    ...   
272  cats_dogs\train\dog\dog_barking_94.wav      1   
273  cats_dogs\train\dog\dog_barking_95.wav      1   
274  cats_dogs\train\dog\dog_barking_96.wav      1   
275  cats_dogs\train\dog\dog_barking_97.wav      1   
276  cats_dogs\train\dog\dog_barking_98.wav      1   

                                                 audio  
0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2    [0.0019836426, 0.002166748, 0.00039672852, -0....  
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
..       